In [2]:
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
survival_df = pd.read_csv("/Users/lindseyvalput/Desktop/Patient-Survival-Model/Resources/cleaned_data_survival_prediction.csv")
survival_df.head()

,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,icu_id,icu_stay_type,icu_type,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,68.0,22.730000,0,Caucasian,M,180.3,Floor,92,admit,CTICU,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
1,77.0,27.420000,0,Caucasian,F,160.0,Floor,90,admit,Med-Surg ICU,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0
2,67.0,27.560000,0,Caucasian,M,190.5,Accident & Emergency,95,admit,Med-Surg ICU,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic,0
3,72.0,28.257052,1,Hispanic,F,154.9,Operating Room / Recovery,113,admit,Med-Surg ICU,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Respiratory,Respiratory,0
4,46.0,25.845717,0,Hispanic,M,167.6,Accident & Emergency,92,admit,CTICU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0


In [4]:
survival_df_features = survival_df[['apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob',
                                  'd1_spo2_min', 'd1_temp_min', 'd1_sysbp_noninvasive_min',
                                  'd1_sysbp_min', 'd1_heartrate_min', 'heart_rate_apache',
                                  'apache_3j_diagnosis', 'pre_icu_los_days','hospital_death']]

In [5]:
X = survival_df_features.drop('hospital_death', axis=1)
y = survival_df_features['hospital_death']

In [6]:
X = pd.get_dummies(X)

### Balance the data set - OVERsampling

In [7]:
#Balance Data - oversampling
ros = RandomOverSampler(random_state=42)

x_ros, y_ros = ros.fit_resample(X, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

Original dataset shape Counter({0: 52044, 1: 4891})
Resample dataset shape Counter({0: 52044, 1: 52044})


In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_ros, y_ros, random_state=45)

In [9]:
clf = AdaBoostClassifier(random_state=1, n_estimators=50, base_estimator=DecisionTreeClassifier(max_depth=2)).fit(X_train, y_train)
print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')

Training Score: 0.7973893884661697
Testing Score: 0.7950195988010145


In [10]:
y_true = y_test
y_pred = clf.predict(X_test)

print('Test Acc: %.3f' % clf.score(X_test, y_test))
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

Test Acc: 0.795
[[10304  2779]
 [ 2555 10384]]
              precision    recall  f1-score   support

           0       0.80      0.79      0.79     13083
           1       0.79      0.80      0.80     12939

    accuracy                           0.80     26022
   macro avg       0.80      0.80      0.80     26022
weighted avg       0.80      0.80      0.80     26022

